In [1]:
try:
    sc.stop()
except:
    pass
#---------------------------#
from pyspark.sql import SparkSession

spark = SparkSession\
    .builder \
    .master('spark://master:7077') \
    .appName('PySpark-Kagggle-AmazonBook_Sample_18months') \
    .getOrCreate()

In [2]:
csvfilePath="hdfs://master:9000/Data_Sample/Sample_1000_amazon-sales-rank-data-for-print-and-kindle-books/amazon_com_extras.csv"

csvDF = spark.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load(csvfilePath).cache()

csvDF.printSchema()

root
 |-- ASIN: string (nullable = true)
 |-- GROUP: string (nullable = true)
 |-- FORMAT: string (nullable = true)
 |-- TITLE: string (nullable = true)
 |-- AUTHOR: string (nullable = true)
 |-- PUBLISHER: string (nullable = true)



In [3]:
HadoopOutPath1 = "hdfs://master:9000/xgm/output/KaggleAmazon_18months02/part-00000"

In [4]:
Out_18m_DF1 = spark.read.options(header='false', inferschema='true',delimiter='\t').csv(HadoopOutPath1)\
    .toDF("ASIN","month_201701","month_201702","month_201703","month_201704","month_201705","month_201706",\
         "month_201707","month_201708","month_201709","month_201710","month_201711","month_201712",\
         "month_201801","month_201802","month_201803","month_201804","month_201805","month_201806",\
         )\
    .cache()

In [5]:
join_DF1 = csvDF.join(Out_18m_DF1,csvDF.ASIN == Out_18m_DF1.ASIN).drop(Out_18m_DF1.ASIN)

In [6]:
join_DF1.show(7)

+----------+-----+---------+--------------------+--------------------+--------------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|      ASIN|GROUP|   FORMAT|               TITLE|              AUTHOR|           PUBLISHER|month_201701|month_201702|month_201703|month_201704|month_201705|month_201706|month_201707|month_201708|month_201709|month_201710|month_201711|month_201712|month_201801|month_201802|month_201803|month_201804|month_201805|month_201806|
+----------+-----+---------+--------------------+--------------------+--------------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|022640014X| book|hard

In [7]:
join_DF1.groupBy("FORMAT").count().show()

+--------------------+-----+
|              FORMAT|count|
+--------------------+-----+
|      kindle edition|    3|
|           paperback|  581|
|           hardcover|  367|
|mass market paper...|   12|
+--------------------+-----+



In [8]:
print (join_DF1.groupBy("FORMAT").count().collect())

[Row(FORMAT='kindle edition', count=3), Row(FORMAT='paperback', count=581), Row(FORMAT='hardcover', count=367), Row(FORMAT='mass market paperback', count=12)]


## 以下是2018年6月各种前5名统计

In [9]:
join_DF1.select("GROUP","TITLE","month_201806").filter("FORMAT = 'kindle edition'").sort("month_201806",ascending=False).show(5)

+------+--------------------+------------+
| GROUP|               TITLE|month_201806|
+------+--------------------+------------+
|kindle|         The Teacher|     1517717|
|kindle|   Our Little Secret|      984472|
|kindle|Romance: Interrac...|      568478|
+------+--------------------+------------+



In [10]:
join_DF1.select("GROUP","TITLE","month_201806").filter("FORMAT = 'paperback'").sort("month_201806",ascending=False).show(5)

+-----+--------------------+------------+
|GROUP|               TITLE|month_201806|
+-----+--------------------+------------+
| book|We're British, In...|    16512689|
| book|Dinner with a Vam...|    15163390|
| book|Small Wars Permit...|    14408970|
| book|   501st (Star Wars)|    13897247|
| book|Game of Thrones (...|    12361751|
+-----+--------------------+------------+
only showing top 5 rows



In [11]:
join_DF1.select("GROUP","TITLE","month_201806").filter("FORMAT = 'hardcover'").sort("month_201806",ascending=False).show(5)

+-----+--------------------+------------+
|GROUP|               TITLE|month_201806|
+-----+--------------------+------------+
| book|Interpreting Quan...|    12411651|
| book|Stirring Slowly: ...|    11193687|
| book|No Cheering In th...|     9862890|
| book|The Mayfair Myste...|     8862649|
| book|Beyond Weird: Why...|     8726490|
+-----+--------------------+------------+
only showing top 5 rows



In [12]:
join_DF1.select("GROUP","TITLE","FORMAT","month_201806").filter("FORMAT = 'mass market paperback'").sort("month_201806",ascending=False).show(5)

+-----+--------------------+--------------------+------------+
|GROUP|               TITLE|              FORMAT|month_201806|
+-----+--------------------+--------------------+------------+
| book|Flowers for Algernon|mass market paper...|      296553|
| book| The Rogue Retrieval|mass market paper...|      151668|
| book|       The Outsiders|mass market paper...|      135661|
| book|The Magicians' Gu...|mass market paper...|      120697|
| book|Patrick: Son of I...|mass market paper...|       45999|
+-----+--------------------+--------------------+------------+
only showing top 5 rows

